In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 超参数
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [2]:
# 数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#CIFAR-10是一个经典的图像分类数据集，包含60,000张32×32像素的彩色图像，分为10个类别（如飞机、汽车、鸟类等），其中50,000张用于训练，10,000张用于测试。
train_dataset = torchvision.datasets.CIFAR10(root='./_asset/data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./_asset/data', train=False,
                                            download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100.0%


nn.ConV2d()
输入四维张量[N, C, H, W]，N为batch大小，C为通道数，H为高，W为宽,
输出四维张量[N, out_channels, out_height, out_width]
实例化需要至少3个参数：输入通道数，输出通道数，卷积核大小

In [6]:
# 定义卷积网络
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 16 * 5 * 5 输出通道数 * conv2 pool后输出的宽 * conv2 pool后输出的高
        self.fc2 = nn.Linear(120, 84)# 中间层神经元数
        self.fc3 = nn.Linear(84, 10)# 输出的类别数

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = self.conv2(x)
        x = self.pool(F.relu(x))
        x = x.view(-1, 16 * 5 * 5) #-1：batch的大小，自动调整
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x





In [7]:
# 创建模型
model = ConvNet().to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        # 后向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 打印结果
        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 2.3156
Epoch [1/4], Step [4000/12500], Loss: 2.2719
Epoch [1/4], Step [6000/12500], Loss: 2.3146
Epoch [1/4], Step [8000/12500], Loss: 2.3219
Epoch [1/4], Step [10000/12500], Loss: 2.1903
Epoch [1/4], Step [12000/12500], Loss: 2.0981
Epoch [2/4], Step [2000/12500], Loss: 2.3341
Epoch [2/4], Step [4000/12500], Loss: 2.5382
Epoch [2/4], Step [6000/12500], Loss: 2.2500
Epoch [2/4], Step [8000/12500], Loss: 2.7287
Epoch [2/4], Step [10000/12500], Loss: 1.7380
Epoch [2/4], Step [12000/12500], Loss: 2.5452
Epoch [3/4], Step [2000/12500], Loss: 1.5898
Epoch [3/4], Step [4000/12500], Loss: 1.5992
Epoch [3/4], Step [6000/12500], Loss: 1.6779
Epoch [3/4], Step [8000/12500], Loss: 2.3820
Epoch [3/4], Step [10000/12500], Loss: 2.5088
Epoch [3/4], Step [12000/12500], Loss: 1.3530
Epoch [4/4], Step [2000/12500], Loss: 2.7410
Epoch [4/4], Step [4000/12500], Loss: 1.8990
Epoch [4/4], Step [6000/12500], Loss: 1.3559
Epoch [4/4], Step [8000/12500], Loss: 1.5290
Epoc

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accuracy} %')
    for i in range(10):
        accuracy = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {accuracy} %')


Accuracy of the network: 46.7 %
Accuracy of plane: 53.6 %
Accuracy of car: 71.9 %
Accuracy of bird: 23.9 %
Accuracy of cat: 22.4 %
Accuracy of deer: 39.5 %
Accuracy of dog: 39.1 %
Accuracy of frog: 56.0 %
Accuracy of horse: 58.4 %
Accuracy of ship: 55.9 %
Accuracy of truck: 46.3 %
